# Responsible ML - Homomorphic Encryption

## Install prerequisites

Before running the notebook, make sure the correct versions of these libraries are installed.

In [ ]:
!pip install encrypted-inference --upgrade

## Setup Azure ML

In the next cell, we create a new Workspace config object using the `<subscription_id>`, `<resource_group_name>`, and `<workspace_name>`. This will fetch the matching Workspace and prompt you for authentication. Please click on the link and input the provided details.

For more information on **Workspace**, please visit: [Microsoft Workspace Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py)

`<subscription_id>` = You can get this ID from the landing page of your Resource Group.

`<resource_group_name>` = This is the name of your Resource Group.

`<workspace_name>` = This is the name of your Workspace.

In [ ]:
from azureml.core.workspace import Workspace
import warnings

warnings.filterwarnings('ignore')

try:    
    ws = Workspace(
        subscription_id = '<subscription_id>', 
        resource_group = '<resource_group>', 
        workspace_name = '<workspace_name>')

    # Writes workspace config file
    ws.write_config()
    
    print('Library configuration succeeded')
except Exception as e:
    print(e)
    print('Workspace not found')

# Homomorphic Encryption

Homomorphic Encryption refers to a new type of encryption technology that allows computation to be directly on encrypted data, without requiring any decryption in the process. 

<img src="./images/encrypted.png" alt="Forest" style="display: inline-block;margin-left: auto;margin-right: auto;width:45%">

## Fetch Model from registry

Next, fetch the latest model from our model registry.

In [ ]:
from azureml.core.model import Model
from scripts.utils import *

tabular = fetch_registered_dataset(ws)
synth_df, Y = prepareDataset(tabular)
X_train, X_test, Y_train, Y_test, A_train, A_test = split_dataset(synth_df, Y)
model = Model(ws, 'loan_approval_grid_model_30')
model.version

## Create managed-endpoints directory

Create a new directory to hold the configuration files for deploying a managed endpoint.

In [ ]:
import os

managed_endpoints = './managed-endpoints'

# Working directory
if not os.path.exists(managed_endpoints):
    os.makedirs(managed_endpoints)
    
if os.path.exists(os.path.join(managed_endpoints,".amlignore")):
  os.remove(os.path.join(managed_endpoints,".amlignore"))

## Create Scoring File

Creating the scoring file is next step before deploying the service. This file is responsible for the actual generation of predictions using the model. The values or scores generated can represent predictions of future values, but they might also represent a likely category or outcome.

The first thing to do in the scoring file is to fetch the model. This is done by calling `Model.get_model_path()` and passing the model name as a parameter.

After the model has been loaded, the function `model.predict()` function should be called to start the scoring process.

For more information on **Machine Learning - Score**, please visit: [Microsoft Machine Learning - Score Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/machine-learning-score)


In [ ]:
%%writefile $managed_endpoints/score.py
import os
import json
import pandas as pd
from azureml.core.model import Model
import joblib
from azure.storage.blob import BlobServiceClient
from encrypted.inference.eiserver import EIServer

def init():
    global model
    # this name is model.id of model that we want to deploy
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "loan_approval_grid_model_30.pkl")
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    
    global server
    server = EIServer(model.coef_, model.intercept_, verbose=True)

def run(raw_data):
    json_properties = json.loads(raw_data)

    key_id = json_properties['key_id']
    conn_str = json_properties['conn_str']
    container = json_properties['container']
    data = json_properties['data']

    # download the Galois keys from blob storage 
    blob_service_client = BlobServiceClient.from_connection_string(conn_str=conn_str)
    blob_client = blob_service_client.get_blob_client(container=container, blob=key_id)
    public_keys = blob_client.download_blob().readall()
    
    result = {}
    # make prediction
    result = server.predict(data, public_keys)

    # you can return any data type as long as it is JSON-serializable
    return result

## Create the environment definition

The following file contains the details of the environment to host the model and code. 

In [ ]:
%%writefile $managed_endpoints/score-new.yml
name: loan-managed-env
channels:
  - conda-forge
dependencies:
  - python=3.7
  - numpy
  - pip
  - scikit-learn==0.22.1
  - scipy
  - pip:
    - azureml-defaults
    - azureml-sdk[notebooks,automl]
    - pandas
    - inference-schema[numpy-support]
    - joblib
    - numpy
    - scipy
    - encrypted-inference==0.9
    - azure-storage-blob

## Define the endpoint configuration
Specific inputs are required to deploy a model on an online endpoint:

1. Model files.
1. The code that's required to score the model.
1. An environment in which your model runs.
1. Settings to specify the instance type and scaling capacity.

In [ ]:
%%writefile $managed_endpoints/endpointconfig.yml
name: loan-managed-endpoint
type: online
auth_mode: key
traffic:
  blue: 100

deployments:
  #blue deployment
  - name: blue
    model: azureml:loan_approval_grid_model_30:1
    code_configuration:
      code:
        local_path: ./
      scoring_script: score.py
    environment: 
      name: loan-managed-env
      version: 1
      path: ./
      conda_file: file:./score-new.yml
      docker:
          image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1
    instance_type: Standard_DS3_v2
    scale_settings:
      scale_type: manual
      instance_count: 1
      min_instances: 1
      max_instances: 2

## Deployment

<img align="center" src="./images/MLOPs-2.gif"/>

## Deploy your managed online endpoint to Azure

This deployment might take up to 15 minutes, depending on whether the underlying environment or image is being built for the first time. Subsequent deployments that use the same environment will finish processing more quickly.

In [ ]:
!az ml endpoint create -g [your resource group name] -w [your AML workspace name] -n loan-managed-endpoint -f ./managed-endpoints/endpointconfig.yml

## Create public and private keys

In order to work with Homomorphic Encryption we need to generate our private and public keys to satisfy the encryption process.

`EILinearRegressionClient` allows us to create a homomorphic encryption based client, and public keys.

To register our training data with our Workspace we need to get the data into the data store. The Workspace will already have a default data store. The function `ws.get_default_datastore()` returns an instance of the data store associated with the Workspace.

For more information on **Datastore**, please visit: [Microsoft Datastore Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore?view=azure-ml-py)

For more information on **How to deploy an encrypted inferencing web service**, please visit: [Microsoft How to deploy an encrypted inferencing web service Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-homomorphic-encryption-seal)


In [ ]:
import os
import azureml.core
from azureml.core import Workspace, Datastore
from encrypted.inference.eiclient import EILinearRegressionClient

# Create a new Encrypted inference client and a new secret key.
edp = EILinearRegressionClient(verbose=True)

public_keys_blob, public_keys_data = edp.get_public_keys()

datastore = ws.get_default_datastore()
container_name = datastore.container_name

# Create a local file and write the keys to it
public_keys = open(public_keys_blob, "wb")
public_keys.write(public_keys_data)
public_keys.close()

# Upload the file to blob store
datastore.upload_files([public_keys_blob])

# Delete the local file
os.remove(public_keys_blob)

In [ ]:
sample_index = 4

print(X_test.iloc[sample_index].to_frame())
inputData = X_test.iloc[sample_index]
sample_data = (X_test.to_numpy())

In [ ]:
raw_data = edp.encrypt(sample_data[sample_index])

## Testing the Service with Encrypted data

Now with test data, we can get it into a suitable format to consume the web service. First an instance of the web service should be obtained by calling the constructor `Webservice()` with the Workspace object and the service name as parameters. 

For more information on **Webservice**, please visit: [Microsoft Webservice Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice?view=azure-ml-py)

In [ ]:
import json

#pass the connection string for blob storage to give the server access to the uploaded public keys 
conn_str_template = 'DefaultEndpointsProtocol={};AccountName={};AccountKey={};EndpointSuffix=core.windows.net'
conn_str = conn_str_template.format(datastore.protocol, datastore.account_name, datastore.account_key)

#build the json 
data = json.dumps({"data": raw_data, "key_id" : public_keys_blob, "conn_str" : conn_str, "container" : container_name })

## Generate a sample request JSON file

Export some test data to a JSON file we can send to the endpoint.

In [ ]:
with open(os.path.join(managed_endpoints, 'sample-request.json'), 'w') as file:
  file.write(data)

## Invoke the endpoint to score data by using your model

You can use either the invoke command or a REST client of your choice to invoke the endpoint and score against it.

In [ ]:
!az ml endpoint invoke -g [your resource group name] -w [your AML workspace name] -n loan-managed-endpoint --request-file ./managed-endpoints/sample-request.json > ./managed_endpoints/sample-response.json

## Decrypting Service Response

The below cell uses the `decrypt()` function to decrypt the response from the deployed service. 

In [ ]:
import numpy as np
import json

eresult = None
with open(os.path.join(managed_endpoints, 'sample-response.json'), 'r') as file:
    eresult = json.loads(json.loads(file.read()))

results = edp.decrypt(eresult)

print ('Decrypted the results ', results)

#Apply argmax to identify the prediction result
prediction = 'Deny'
if results[0] > 0:
    prediction = 'Approve'

actual = 'Deny'
if Y_test[sample_index] == 1:
    actual = 'Approve'

print ( ' Prediction : ', prediction)
print( 'Actual : ', actual)

## Optional: Deploy to Azure Container Instance

In [ ]:
!cp $managed_endpoints/score.py ./score.py

## Deployment dependencies

The first step is to define the dependencies that are needed for the service to run and they are defined by calling `CondaDependencies.create()`. This create function will receive as parameters the pip and conda packages to install on the remote machine. Secondly, the output of this function is persisted into a `.yml` file that will be leveraged later on the process.

Now it's time to create a `InferenceConfig` object by calling its constructor and passing the runtime type, the path to the `entry_script` (score.py), and the `conda_file` (the previously created file that holds the environment dependencies).

The `CondaDependencies.create()` function initializes a new CondaDependencies object.

For more information on **CondaDependencies**, please visit: [Microsoft CondaDependencies Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py)

For more information on **InferenceConfig**, please visit: [Microsoft InferenceConfig Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py)

In [ ]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.conda_dependencies import CondaDependencies

azureml_pip_packages = ['azureml-defaults', 'azureml-contrib-interpret', 'azureml-core', 'azureml-telemetry',
                        'azureml-interpret', 'azureml-dataprep','azureml-dataprep[fuse,pandas]','joblib',
                        'matplotlib','scikit-learn==0.22.1','seaborn','fairlearn','encrypted-inference==0.9','azure-storage-blob']

# Define dependencies needed in the remote environment
myenv = CondaDependencies.create(pip_packages=azureml_pip_packages)

# Write dependencies to yml file
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

# Create an inference config object based on the score.py and myenv.yml from previous steps
inference_config = InferenceConfig(runtime= "python",
                                    entry_script="score.py",
                                    conda_file="myenv.yml")

## Deploy model to Azure Container Instance

In order to deploy the to an Azure Container Instance, the function `Model.deploy()` should be called, passing along the workspace object, service name and list of models to deploy.

`Webservice` defines base functionality for deploying models as web service endpoints in Azure Machine Learning. Webservice constructor is used to retrieve a cloud representation of a Webservice object associated with the provided Workspace.

The `AciWebService` represents a machine learning model deployed as a web service endpoint on Azure Container Instances. A deployed service is created from a model, script, and associated files. The resulting web service is a load-balanced, HTTP endpoint with a REST API. You can send data to this API and receive the prediction returned by the model.


For more information on **Model**, please visit: [Microsoft Model Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py)

For more information on **Webservice**, please visit: [Microsoft Webservice Class Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice(class)?view=azure-ml-py)

For more information on **AciWebservice**, please visit: [Microsoft AciWebservice Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.aci.aciwebservice?view=azure-ml-py)

**Note:** Please wait for the execution of the cell to finish before moving forward.

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.model import Model

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2,
                                               description = "Loan approval service")

service_name_aci = 'loan-approval-aci'
print(service_name_aci)

try:
    aci_service = Webservice(ws, service_name_aci)
    print(aci_service.state)
except WebserviceException:
    aci_service = Model.deploy(ws, service_name_aci, [model], inference_config, aciconfig)
    aci_service.wait_for_deployment(True)
    print(aci_service.state)

## Testing the Service with Encrypted data

Now with test data, we can get it into a suitable format to consume the web service. First an instance of the web service should be obtained by calling the constructor `Webservice()` with the Workspace object and the service name as parameters. 

For more information on **Webservice**, please visit: [Microsoft Webservice Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice?view=azure-ml-py)

In [ ]:
import json
from azureml.core import Webservice

service = Webservice(ws, service_name_aci)

#pass the connection string for blob storage to give the server access to the uploaded public keys 
conn_str_template = 'DefaultEndpointsProtocol={};AccountName={};AccountKey={};EndpointSuffix=core.windows.net'
conn_str = conn_str_template.format(datastore.protocol, datastore.account_name, datastore.account_key)

#build the json 
data = json.dumps({"data": raw_data, "key_id" : public_keys_blob, "conn_str" : conn_str, "container" : container_name })
data = bytes(data, encoding='ASCII')

print ('Making an encrypted inference web service call ')
eresult = service.run(input_data=data)

print ('Received encrypted inference results')
print (f'Encrypted results: ...', eresult[0][0:100], '...')

## Decrypting Service Response

The below cell uses the `decrypt()` function to decrypt the response from the deployed ACI Service. 

In [ ]:
import numpy as np 

results = edp.decrypt(eresult)

print ('Decrypted the results ', results)

#Apply argmax to identify the prediction result
prediction = 'Deny'
if results[0] > 0:
    prediction = 'Approve'

actual = 'Deny'
if Y_test[sample_index] == 1:
    actual = 'Approve'

print ( ' Prediction : ', prediction)
print( 'Actual : ', actual)